In [1]:
from textwrap import wrap
import os

!pip install keras-cv==0.4.0 -q
#!pip install git+https://github.com/keras-team/keras-cv.git tensorflow --upgrade
!pip install tensorflow==2.10.0
!pip install tensorflow_datasets
!pip install matplotlib
!pip install pandas
!pip install charset-normalizer==3.1.0
!pip install -U tensorflow-addons
import tensorflow_addons as tfa

import keras_cv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.experimental.numpy as tnp
from keras_cv.models.stable_diffusion.clip_tokenizer import SimpleTokenizer
from keras_cv.models.stable_diffusion.diffusion_model import DiffusionModel
from keras_cv.models.stable_diffusion.image_encoder import ImageEncoder
from keras_cv.models.stable_diffusion.noise_scheduler import NoiseScheduler
from keras_cv.models.stable_diffusion.text_encoder import TextEncoder
from tensorflow import keras
tf.keras.backend.set_floatx('float32')

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-datasets 4.9.3 requires protobuf>=3.20, but you have protobuf 3.19.6 which is incompatible.
tensorflow-metadata 1.14.0 requires protobuf<4.21,>=3.20.3, but you have protobuf 3.19.6 which is incompatible.



  Using cached protobuf-3.19.6-cp39-cp39-win_amd64.whl (895 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Using cached protobuf-4.25.1-cp39-cp39-win_amd64.whl.metadata (541 bytes)
  Using cached protobuf-3.20.3-cp39-cp39-win_amd64.whl (904 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.6
    Uninstalling protobuf-3.19.6:
      Successfully uninstalled protobuf-3.19.6


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorboard 2.10.1 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which is incompatible.
tensorflow 2.10.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which is incompatible.


c:\Users\Administrator\anaconda3\envs\sd_keras\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
c:\Users\Administrator\anaconda3\envs\sd_keras\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.12.0 and strictly below 2.15.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.10.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do no

You do not have pycocotools installed, so KerasCV pycoco metrics are not available. Please run `pip install pycocotools`.


In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  2


In [3]:
data_path = 'D:/dataset'

data_frame = pd.read_csv(os.path.join(data_path, "data.csv"))

data_frame["image_path"] = data_frame["image_path"].apply(
    lambda x: os.path.join(data_path, x)
)
data_frame.head()

,image_path,caption
0,D:/dataset\100023.jpg,Nước ngọt CocaCola
1,D:/dataset\100023.jpg,Thức uống có gas
2,D:/dataset\100033.jpg,Chiếc xe ô tô màu trắng
3,D:/dataset\100033.jpg,Xe ô tô
4,D:/dataset\100113.jpg,Món ăn sushi của Nhật Bản


In [4]:
!pip install transformers
!pip3 install tokenizers

In [5]:
import torch
from transformers import AutoModel, AutoTokenizer
PADDING_TOKEN = 64000
MAX_PROMPT_LENGTH = 256

tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2", add_special_tokens=False)

def process_text(caption):
    tokens = tokenizer.encode(caption)
    tokens = torch.tensor([tokens + [PADDING_TOKEN] * (MAX_PROMPT_LENGTH - len(tokens))])
    return tokens

# Collate the tokenized captions into an array.
tokenized_texts = np.empty((len(data_frame), MAX_PROMPT_LENGTH))

all_captions = list(data_frame["caption"].values)

for i, caption in enumerate(all_captions):
    tokenized_texts[i] = process_text(caption)

In [6]:
RESOLUTION = 256
AUTO = tf.data.AUTOTUNE
#POS_IDS = tf.convert_to_tensor([list(range(MAX_PROMPT_LENGTH))], dtype=tf.int64)

augmenter = keras.Sequential(
    layers=[
        keras_cv.layers.CenterCrop(RESOLUTION, RESOLUTION),
        #keras_cv.layers.RandomFlip(),
        tf.keras.layers.Rescaling(scale=1.0 / 127.5, offset=-1),
    ]
)
text_encoder = AutoModel.from_pretrained("vinai/phobert-base-v2")
# def process_image(image_path, tokenized_text):
#     image = tf.io.read_file(image_path)
#     image = tf.io.decode_png(image, 3)
#     image = tf.image.resize(image, (RESOLUTION, RESOLUTION))
#     return image, tokenized_text

def process_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.io.decode_png(image, 3)
    image = tf.image.resize(image, (RESOLUTION, RESOLUTION))
    return image

# def apply_augmentation(image_batch, token_batch):
#     return augmenter(image_batch), token_batch

def apply_augmentation(image_batch):
    return augmenter(image_batch)

def run_text_encoder(image_batch, token_batch):
    return (
        image_batch,
        token_batch,
        text_encoder(token_batch),
    )


def prepare_dict(image_batch, token_batch, encoded_text_batch):
    return {
        "images": image_batch,
        "tokens": token_batch,
        "encoded_text": encoded_text_batch,
    }

def prepare_dataset(image_paths, tokenized_texts, batch_size=1):
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, tokenized_texts.astype(np.int32)))
    dataset = dataset.shuffle(batch_size * 10)
    # dataset = dataset.map(process_image, num_parallel_calls=AUTO).batch(batch_size)
    # dataset = dataset.map(apply_augmentation, num_parallel_calls=AUTO)
    # dataset = dataset.map(run_text_encoder, num_parallel_calls=AUTO)
    # dataset = dataset.map(prepare_dict, num_parallel_calls=AUTO)
    return dataset.prefetch(AUTO)

Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
with tf.device('/CPU:0'):
    # Prepare the dataset.
    BATCH_SIZE = 4
    tf_training_dataset = prepare_dataset(
        np.array(data_frame["image_path"]), tokenized_texts, batch_size=4
    )
    list_dict = []
    for element in tf_training_dataset:
        image = process_image(element[0].numpy())
        image_augmentation = apply_augmentation(image)
        tf_element = torch.tensor([np.array(element[1].numpy())])
        encoded_text_batch = text_encoder(tf_element)
        dict_dt = prepare_dict(image_augmentation, element[1], tf.convert_to_tensor(encoded_text_batch.last_hidden_state[0].detach().numpy()))
        list_dict.append(dict_dt)

    training_dataset = tf.data.Dataset.from_tensor_slices(pd.DataFrame.from_dict(list_dict).to_dict(orient="list"))
    training_dataset = training_dataset.shuffle(BATCH_SIZE * 10)
    training_dataset = training_dataset.batch(BATCH_SIZE)

    # Take a sample batch and investigate.
    sample_batch = next(iter(training_dataset))

    for k in sample_batch:
        print(k, sample_batch[k].shape)

C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_3776\532460767.py:11: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\b\abs_2112s1s0to\croot\pytorch-select_1700158736573\work\torch\csrc\utils\tensor_new.cpp:264.)
  tf_element = torch.tensor([np.array(element[1].numpy())])


In [ ]:
class Trainer(tf.keras.Model):
    # Reference:
    # https://github.com/huggingface/diffusers/blob/main/examples/text_to_image/train_text_to_image.py

    def __init__(
        self,
        diffusion_model,
        vae,
        noise_scheduler,
        use_mixed_precision=False,
        max_grad_norm=1.0,
        **kwargs
    ):
        super().__init__(**kwargs)

        self.diffusion_model = diffusion_model
        self.vae = vae
        self.noise_scheduler = noise_scheduler
        self.max_grad_norm = max_grad_norm

        self.use_mixed_precision = use_mixed_precision
        self.vae.trainable = False

    def train_step(self, inputs):
        images = inputs["images"]
        encoded_text = inputs["encoded_text"]
        batch_size = tf.shape(images)[0]

        with tf.GradientTape() as tape:
            # Project image into the latent space and sample from it.
            latents = self.sample_from_encoder_outputs(self.vae(images, training=False))
            # Know more about the magic number here:
            # https://keras.io/examples/generative/fine_tune_via_textual_inversion/
            latents = latents * 0.18215

            # Sample noise that we'll add to the latents.
            noise = tf.random.normal(tf.shape(latents))

            # Sample a random timestep for each image.
            timesteps = tnp.random.randint(
                0, self.noise_scheduler.train_timesteps, (batch_size,)
            )

            # Add noise to the latents according to the noise magnitude at each timestep
            # (this is the forward diffusion process).
            noisy_latents = self.noise_scheduler.add_noise(
                tf.cast(latents, dtype=tf.float32), tf.cast(noise, dtype=tf.float32), timesteps
            )

            # Get the target for loss depending on the prediction type
            # just the sampled noise for now.
            target = noise  # noise_schedule.predict_epsilon == True

            # Predict the noise residual and compute loss.
            timestep_embedding = tf.map_fn(
                lambda t: self.get_timestep_embedding(t), timesteps, dtype=tf.float32
            )
            timestep_embedding = tf.squeeze(timestep_embedding, 1)
            model_pred = self.diffusion_model(
                [noisy_latents, timestep_embedding, encoded_text], training=True
            )
            loss = self.compiled_loss(target, model_pred)
            if self.use_mixed_precision:
                loss = self.optimizer.get_scaled_loss(loss)

        # Update parameters of the diffusion model.
        trainable_vars = self.diffusion_model.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)
        if self.use_mixed_precision:
            gradients = self.optimizer.get_unscaled_gradients(gradients)
        gradients = [tf.clip_by_norm(g, self.max_grad_norm) for g in gradients]
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        return {m.name: m.result() for m in self.metrics}

    def get_timestep_embedding(self, timestep, dim=320, max_period=10000):
        half = dim // 2
        log_max_period = tf.math.log(tf.cast(max_period, tf.float32))
        freqs = tf.math.exp(
            -log_max_period * tf.range(0, half, dtype=tf.float32) / half
        )
        args = tf.convert_to_tensor([timestep], dtype=tf.float32) * freqs
        embedding = tf.concat([tf.math.cos(args), tf.math.sin(args)], 0)
        embedding = tf.reshape(embedding, [1, -1])
        return embedding

    def sample_from_encoder_outputs(self, outputs):
        mean, logvar = tf.split(outputs, 2, axis=-1)
        logvar = tf.clip_by_value(logvar, -30.0, 20.0)
        std = tf.exp(0.5 * logvar)
        sample = tf.random.normal(tf.shape(mean), dtype=mean.dtype)
        return mean + std * sample

    def save_weights(self, filepath, overwrite=True, save_format=None, options=None):
        # Overriding this method will allow us to use the `ModelCheckpoint`
        # callback directly with this trainer class. In this case, it will
        # only checkpoint the `diffusion_model` since that's what we're training
        # during fine-tuning.
        self.diffusion_model.save_weights(
            filepath=filepath,
            overwrite=overwrite,
            save_format=save_format,
            options=options,
        )

In [ ]:
# Enable mixed-precision training if the underlying GPU has tensor cores.
with tf.device('/GPU:0'):
    USE_MP = True
    if USE_MP:
        keras.mixed_precision.set_global_policy("float32")

    image_encoder = ImageEncoder(RESOLUTION, RESOLUTION, download_weights=False)
    diffusion_ft_trainer = Trainer(
        diffusion_model=DiffusionModel(RESOLUTION, RESOLUTION, MAX_PROMPT_LENGTH, download_weights=False),
        # Remove the top layer from the encoder, which cuts off the variance and only
        # returns the mean.
        vae=tf.keras.Model(
            image_encoder.input,
            image_encoder.layers[-2].output,
        ),
        noise_scheduler=NoiseScheduler(),
        use_mixed_precision=USE_MP,
    )

    # These hyperparameters come from this tutorial by Hugging Face:
    # https://huggingface.co/docs/diffusers/training/text2image
    lr = 1e-5
    beta_1, beta_2 = 0.9, 0.999
    weight_decay = 1e-2
    epsilon = 1e-08

    optimizer = tf.keras.optimizers.experimental.AdamW(
        learning_rate=lr,
        weight_decay=weight_decay,
        beta_1=beta_1,
        beta_2=beta_2,
        epsilon=epsilon,
    )
    #optimizer_s = tf.keras.mixed_precision.LossScaleOptimizer(optimizer)
    diffusion_ft_trainer.compile(optimizer='adam', loss="mse", metrics=['accuracy'])

In [ ]:
from tqdm.keras import TqdmCallback
with tf.device('/GPU:1'):
    epochs = 20
    ckpt_path = "finetuned_stable_diffusion.h5"
    ckpt_callback = tf.keras.callbacks.ModelCheckpoint(
        ckpt_path,
        save_weights_only=True,
        monitor="loss",
        mode="min",
        save_best_only=False,
        save_freq='epoch',
    )
    diffusion_ft_trainer.fit(training_dataset, epochs=epochs, verbose=0, callbacks=[ckpt_callback,TqdmCallback(verbose=1)])